In [1]:
import pymc3
import umap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
import os
from scipy.stats import zscore
import pickle
%matplotlib inline

# Read in csv files

In [12]:
!ls 'fcs_output'/

ls: cannot access 'fcs_output/': No such file or directory


In [14]:
DATA_DIR = "/data/isshamie/time_neutrophils/fcs output/"

In [18]:
def parse_file(f):
    vals = os.path.basename(f).replace(".csv","").replace(" ", "").split("-")
    d = dict()
    d["Timepoint"] = int(vals[-1].replace("Timepoint",""))
    d["Stimuli"] = vals[1][0]
    d["Sample"] = int(vals[1][1:])
    return d



In [ ]:
def concat_data(files):
    columns = ["Timepoint","Stimuli","Sample", "Cell Size","Cell Circularity","Cell Aspect Ratio","Cell Tracker Intensity","PI Intensity","AnexinV Intensity"]
    data_df = pd.DataFrame(columns=columns)
    for i in files:
        d = parse_file(i)
        curr_df = pd.read_csv(i)
        curr_df["Timepoint"] = d["Timepoint"]
        curr_df["Stimuli"] = d["Stimuli"]
        curr_df["Sample"] = d["Sample"]
        curr_df.index = curr_df.apply(lambda x:  str(x.name) +"_" + str(x["Sample"]) + x["Stimuli"] + str(x["Timepoint"]) ,axis=1)
        data_df = pd.concat((data_df,curr_df),sort=False)
    return data_df

In [ ]:
columns = ["Timepoint","Stimuli","Sample", "Cell Size","Cell Circularity","Cell Aspect Ratio","Cell Tracker Intensity","PI Intensity","AnexinV Intensity"]

data_df = pd.DataFrame(columns=columns)

for i in glob.glob(os.path.join(DATA_DIR, "*csv")):
    print(i)
    d = parse_file(i)
    curr_df = pd.read_csv(i)
    curr_df["Timepoint"] = d["Timepoint"]
    curr_df["Stimuli"] = d["Stimuli"]
    curr_df["Sample"] = d["Sample"]
    curr_df.index = curr_df.apply(lambda x:  str(x.name) +"_" + str(x["Sample"]) + x["Stimuli"] + str(x["Timepoint"]) ,axis=1)
    data_df = pd.concat((data_df,curr_df),sort=False)
    

/data/isshamie/time_neutrophils/fcs output/PAD4 - G02 - Timepoint3.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G11 - Timepoint30.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H01 - Timepoint22.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A08 - Timepoint3.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B12 - Timepoint2.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B01 - Timepoint36.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H08 - Timepoint8.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B11 - Timepoint16.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F04 - Timepoint14.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E05 - Timepoint20.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F05 - Timepoint19.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G08 - Timepoint46.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D10 - Timepoint8.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F10 - Timepoint41.c

/data/isshamie/time_neutrophils/fcs output/PAD4 - F11 - Timepoint10.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A03 - Timepoint15.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E02 - Timepoint32.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B03 - Timepoint5.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A10 - Timepoint19.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E05 - Timepoint36.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F08 - Timepoint27.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G10 - Timepoint46.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H06 - Timepoint2.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G08 - Timepoint40.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G01 - Timepoint36.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F02 - Timepoint26.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B02 - Timepoint14.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G05 - Timepoint2

/data/isshamie/time_neutrophils/fcs output/PAD4 - F01 - Timepoint15.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C08 - Timepoint42.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E03 - Timepoint25.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C12 - Timepoint45.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A08 - Timepoint14.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E06 - Timepoint16.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D08 - Timepoint12.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G02 - Timepoint14.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C04 - Timepoint19.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H07 - Timepoint12.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F09 - Timepoint7.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C02 - Timepoint29.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B01 - Timepoint10.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F02 - Timepoint

/data/isshamie/time_neutrophils/fcs output/PAD4 - G12 - Timepoint40.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D01 - Timepoint33.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B10 - Timepoint15.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A04 - Timepoint44.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A09 - Timepoint16.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F12 - Timepoint27.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D05 - Timepoint18.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B04 - Timepoint9.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C11 - Timepoint33.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E10 - Timepoint20.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C10 - Timepoint43.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D08 - Timepoint4.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F07 - Timepoint10.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H08 - Timepoint3

/data/isshamie/time_neutrophils/fcs output/PAD4 - D03 - Timepoint43.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F03 - Timepoint5.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G04 - Timepoint16.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G08 - Timepoint21.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C07 - Timepoint32.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F11 - Timepoint21.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C04 - Timepoint43.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E02 - Timepoint11.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H01 - Timepoint3.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C04 - Timepoint4.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G03 - Timepoint33.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G08 - Timepoint44.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H04 - Timepoint29.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F11 - Timepoint44

...

/data/isshamie/time_neutrophils/fcs output/PAD4 - A09 - Timepoint3.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B06 - Timepoint21.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F12 - Timepoint37.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B12 - Timepoint40.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F06 - Timepoint15.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C05 - Timepoint11.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F04 - Timepoint35.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C12 - Timepoint14.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C12 - Timepoint20.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C06 - Timepoint26.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G10 - Timepoint40.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G11 - Timepoint24.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D10 - Timepoint9.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E01 - Timepoint9

/data/isshamie/time_neutrophils/fcs output/PAD4 - C11 - Timepoint39.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A10 - Timepoint37.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E08 - Timepoint32.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B09 - Timepoint9.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C04 - Timepoint39.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G04 - Timepoint10.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G03 - Timepoint1.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G01 - Timepoint32.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F03 - Timepoint2.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B05 - Timepoint35.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B08 - Timepoint35.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C05 - Timepoint12.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A02 - Timepoint35.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B03 - Timepoint7.

/data/isshamie/time_neutrophils/fcs output/PAD4 - D11 - Timepoint24.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F10 - Timepoint4.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F07 - Timepoint6.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B05 - Timepoint17.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D07 - Timepoint44.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H09 - Timepoint12.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F11 - Timepoint8.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F07 - Timepoint41.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A09 - Timepoint12.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G11 - Timepoint34.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D10 - Timepoint34.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C11 - Timepoint41.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H03 - Timepoint33.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E08 - Timepoint25

/data/isshamie/time_neutrophils/fcs output/PAD4 - E10 - Timepoint26.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C09 - Timepoint7.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A03 - Timepoint5.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B06 - Timepoint16.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A01 - Timepoint3.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F05 - Timepoint7.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A10 - Timepoint27.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A02 - Timepoint12.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G03 - Timepoint29.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A02 - Timepoint23.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E07 - Timepoint34.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F09 - Timepoint21.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - F03 - Timepoint19.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H03 - Timepoint19.

/data/isshamie/time_neutrophils/fcs output/PAD4 - E05 - Timepoint27.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H03 - Timepoint36.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G01 - Timepoint21.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B04 - Timepoint6.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - A01 - Timepoint8.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G04 - Timepoint38.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B12 - Timepoint34.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E03 - Timepoint38.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - E05 - Timepoint3.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - C10 - Timepoint28.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - D02 - Timepoint28.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - G03 - Timepoint15.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - H05 - Timepoint40.csv
/data/isshamie/time_neutrophils/fcs output/PAD4 - B10 - Timepoint27

In [38]:
data_df

,Timepoint,Stimuli,Sample,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,3,G,2,11750,113.2539,2.8375,21192.0999,2951.6684,10590.6212
1_2G3,3,G,2,61,512.9727,61.0000,17836.5246,2907.2951,8491.5246
2_2G3,3,G,2,12,2565.8500,12.0000,18353.1667,2925.5833,8690.5833
3_2G3,3,G,2,23,1351.6133,23.0000,19101.6087,2961.4783,8673.8261
4_2G3,3,G,2,2,10000.0000,2.0000,19001.0000,2935.5000,8735.5000
...,...,...,...,...,...,...,...,...,...
4592_4B46,46,B,4,216,9413.2491,1.0487,14314.2315,4481.2037,27472.5972
4593_4B46,46,B,4,370,4905.5990,3.0213,12515.4270,6447.3568,29679.3784
4594_4B46,46,B,4,157,8635.1936,1.4828,14121.9427,15047.1401,26821.1465
4595_4B46,46,B,4,19,10000.0000,1.2733,20725.6316,46356.0526,26550.7895


In [37]:
data_df.to_csv("fc.tsv",sep="\t")

## Standardize all variables

In [40]:
data_df.drop(["Sample", "Timepoint","Stimuli"],axis=1)

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,11750,113.2539,2.8375,21192.0999,2951.6684,10590.6212
1_2G3,61,512.9727,61.0000,17836.5246,2907.2951,8491.5246
2_2G3,12,2565.8500,12.0000,18353.1667,2925.5833,8690.5833
3_2G3,23,1351.6133,23.0000,19101.6087,2961.4783,8673.8261
4_2G3,2,10000.0000,2.0000,19001.0000,2935.5000,8735.5000
...,...,...,...,...,...,...
4592_4B46,216,9413.2491,1.0487,14314.2315,4481.2037,27472.5972
4593_4B46,370,4905.5990,3.0213,12515.4270,6447.3568,29679.3784
4594_4B46,157,8635.1936,1.4828,14121.9427,15047.1401,26821.1465
4595_4B46,19,10000.0000,1.2733,20725.6316,46356.0526,26550.7895


In [41]:
metadata=data_df[["Sample", "Timepoint","Stimuli"]]
data_df=data_df.drop(["Sample", "Timepoint","Stimuli"],axis=1)

### First 5 are wt, next 6 are Pad4ko
Plate layout	wt	wt	wt	wt	wt	PAD4ko	PAD4ko	PAD4ko	PAD4ko	PAD4ko	PAD4ko	PAD4ko  
DMSO												
zVD												
BPT												
BPT/zVD												
BPT+Nec1s												
BPT/zVD+Nec1s												
Nec1s												
PIK75												


In [34]:
Stimuli_Names = {"A":"DMSO","B":"zVD", "C":"BPT", "D":"BPT_zVD",
                 "E":"BPT_Nec1s", "F":"BPT_zVD_Nec1s","G": "Nec1s","H":"PIK75"}

metadata["Stimuli Names"] = metadata["Stimuli"].map(Stimuli_Names)
metadata

,Timepoint,Stimuli,Sample,Genotype,Stimuli Names
0_2G3,3,G,2,WT,Nec1s
1_2G3,3,G,2,WT,Nec1s
2_2G3,3,G,2,WT,Nec1s
3_2G3,3,G,2,WT,Nec1s
4_2G3,3,G,2,WT,Nec1s
...,...,...,...,...,...
4592_4B46,46,B,4,WT,zVD
4593_4B46,46,B,4,WT,zVD
4594_4B46,46,B,4,WT,zVD
4595_4B46,46,B,4,WT,zVD


In [24]:
metadata["Genotype"] = "WT"
metadata.loc[metadata["Sample"].astype(int) >= 6,"Genotype"] = "Pad4KO"

In [35]:
metadata.to_csv("../data/meta.tsv",sep="\t")

In [19]:
metadata = pd.read_csv("../meta.tsv",sep="\t", index_col=0)
metadata

,Timepoint,Stimuli,Sample
0_2G3,3,G,2
1_2G3,3,G,2
2_2G3,3,G,2
3_2G3,3,G,2
4_2G3,3,G,2
...,...,...,...
4592_4B46,46,B,4
4593_4B46,46,B,4
4594_4B46,46,B,4
4595_4B46,46,B,4


In [42]:
data_df

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,11750,113.2539,2.8375,21192.0999,2951.6684,10590.6212
1_2G3,61,512.9727,61.0000,17836.5246,2907.2951,8491.5246
2_2G3,12,2565.8500,12.0000,18353.1667,2925.5833,8690.5833
3_2G3,23,1351.6133,23.0000,19101.6087,2961.4783,8673.8261
4_2G3,2,10000.0000,2.0000,19001.0000,2935.5000,8735.5000
...,...,...,...,...,...,...
4592_4B46,216,9413.2491,1.0487,14314.2315,4481.2037,27472.5972
4593_4B46,370,4905.5990,3.0213,12515.4270,6447.3568,29679.3784
4594_4B46,157,8635.1936,1.4828,14121.9427,15047.1401,26821.1465
4595_4B46,19,10000.0000,1.2733,20725.6316,46356.0526,26550.7895


In [51]:
data_df_z = data_df.apply(zscore,axis=0)

In [54]:
pickle.dump(data_df_z, open("fc_z.p","wb"))